TODO. add deepnet and new features from:  
https://github.com/abhishekkrthakur/is_that_a_duplicate_quora_question

In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.cross_validation import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials 
import os

import xgboost as xgb


/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> averaged_perceptron_tagger
        /home/ubuntu/nltk_data...
      Unzipping taggers/averaged_perceptron_tagger.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

cleaning, feature engineering, commonize feature train-test

In [2]:
def load_csv():
    global train, test
    train = pd.read_csv('./data/train.csv') #remove limit
    test = pd.read_csv('./data/test.csv') #remove limit
    
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

def get_weight(count, eps=500, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

def common_words():
    global weights
    train_qs = pd.Series(train['question1'].tolist() + train['question2'].tolist()).astype(str)
    test_qs = pd.Series(test['question1'].tolist() + test['question2'].tolist()).astype(str)
    words = (" ".join(train_qs)).lower().split() + (" ".join(test_qs)).lower().split()
    counts = Counter(words)
    weights = {word: get_weight(count) for word, count in counts.items()}

def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

def get_unigrams(que):
    return [word for word in nltk.word_tokenize(que.lower()) if word not in stops]

def get_common_unigrams(row):
    return len( set(row["unigrams_ques1"]).intersection(set(row["unigrams_ques2"])) )

def get_common_unigram_ratio(row):
    return float(row["zunigrams_common_count"]) / max(len( set(row["unigrams_ques1"]).union(set(row["unigrams_ques2"])) ),1)

def get_bigrams(que):
    return [i for i in nltk.ngrams(que, 2)]

def get_common_bigrams(row):
    return len( set(row["bigrams_ques1"]).intersection(set(row["bigrams_ques2"])) )

def get_common_bigram_ratio(row):
    return float(row["zbigrams_common_count"]) / max(len( set(row["bigrams_ques1"]).union(set(row["bigrams_ques2"])) ),1)


In [3]:
def feats_train():
    train['test'] = train.question1.map(lambda x: [t for  t in nltk.pos_tag(nltk.word_tokenize(str(x).lower()))])
    train['test1'] = train.question2.map(lambda x: [t for  t in nltk.pos_tag(nltk.word_tokenize(str(x).lower()))])

    train['question1_nouns'] = train.test.map(lambda x: [w for w, t in x if t == 'NN'])
    train['question2_nouns'] = train.test1.map(lambda x: [w for w, t in x if t == 'NN'])

    train['question1_proper_nouns'] = train.test.map(lambda x: [w for w, t in x if t[:3] in ['NNP']])
    train['question2_proper_nouns'] = train.test1.map(lambda x: [w for w, t in x if t[:3] in ['NNP']])

    train['question1_verbs'] = train.test.map(lambda x: [w for w, t in x if t[:2] in ['VB']])
    train['question2_verbs'] = train.test1.map(lambda x: [w for w, t in x if t[:2] in ['VB']])

    train['question1_jj'] = train.test.map(lambda x: [w for w, t in x if t[:2] in ['JJ']])
    train['question2_jj'] = train.test1.map(lambda x: [w for w, t in x if t[:2] in ['JJ']])

    train['question1_rb'] = train.test.map(lambda x: [w for w, t in x if t[:2] in ['RB']])
    train['question2_rb'] = train.test1.map(lambda x: [w for w, t in x if t[:2] in ['RB']])

    train['question1_cc'] = train.test.map(lambda x: [w for w, t in x if t == 'CC'])
    train['question2_cc'] = train.test1.map(lambda x: [w for w, t in x if t == 'CC'])

    train['question1_cd'] = train.test.map(lambda x: [w for w, t in x if t == 'CD'])
    train['question2_cd'] = train.test1.map(lambda x: [w for w, t in x if t == 'CD'])

    train['question1_dt'] = train.test.map(lambda x: [w for w, t in x if t == 'DT'])
    train['question2_dt'] = train.test1.map(lambda x: [w for w, t in x if t == 'DT'])

    train['question1_ex'] = train.test.map(lambda x: [w for w, t in x if t == 'EX'])
    train['question2_ex'] = train.test1.map(lambda x: [w for w, t in x if t == 'EX'])

    train['question1_ls'] = train.test.map(lambda x: [w for w, t in x if t == 'LS'])
    train['question2_ls'] = train.test1.map(lambda x: [w for w, t in x if t == 'LS'])

    train['question1_wdt'] = train.test.map(lambda x: [w for w, t in x if t == 'WDT'])
    train['question2_wdt'] = train.test1.map(lambda x: [w for w, t in x if t == 'WDT'])

    train['z_word_len1'] = train.question1.map(lambda x: len(str(x).split()))
    train['z_word_len2'] = train.question2.map(lambda x: len(str(x).split()))
    train['z_noun_match'] = train.apply(lambda r: sum([1 for w in r.question1_nouns if w in r.question2_nouns]), axis=1)
    train['z_verb_match'] = train.apply(lambda r: sum([1 for w in r.question1_verbs if w in r.question2_verbs]), axis=1)
    train['z_prop_noun_match'] = train.apply(lambda r: sum([1 for w in r.question1_proper_nouns if w in r.question2_proper_nouns]), axis=1)
    train['z_cc_match'] = train.apply(lambda r: sum([1 for w in r.question1_cc if w in r.question2_cc]), axis=1)
    train['z_cd_match'] = train.apply(lambda r: sum([1 for w in r.question1_cd if w in r.question2_cd]), axis=1)
    train['z_dt_match'] = train.apply(lambda r: sum([1 for w in r.question1_dt if w in r.question2_dt]), axis=1)
    train['z_jj_match'] = train.apply(lambda r: sum([1 for w in r.question1_jj if w in r.question2_jj]), axis=1)
    train['z_rb_match'] = train.apply(lambda r: sum([1 for w in r.question1_rb if w in r.question2_rb]), axis=1)
    train['z_ex_match'] = train.apply(lambda r: sum([1 for w in r.question1_ex if w in r.question2_ex]), axis=1)
    train['z_ls_match'] = train.apply(lambda r: sum([1 for w in r.question1_ls if w in r.question2_ls]), axis=1)
    train['z_wdt_match'] = train.apply(lambda r: sum([1 for w in r.question1_wdt if w in r.question2_wdt]), axis=1)


    #train['nr_nouns'] = train.question1.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    train['total_nouns'] = train['question1_nouns'] + train['question2_nouns']
    train['total_verbs'] = train['question1_verbs'] + train['question2_verbs']
    train['total_proper_nouns'] = train['question1_proper_nouns'] + train['question2_proper_nouns']
    train['total_cc'] = train['question1_cc'] + train['question2_cc']
    train['total_cd'] = train['question1_cd'] + train['question2_cd']
    train['total_dt'] = train['question1_dt'] + train['question2_dt']
    train['total_jj'] = train['question1_jj'] + train['question2_jj']
    train['total_rb'] = train['question1_rb'] + train['question2_rb']
    train['total_ex'] = train['question1_ex'] + train['question2_ex']
    train['total_ls'] = train['question1_ls'] + train['question2_ls']
    train['total_wdt'] = train['question1_wdt'] + train['question2_wdt']

    train['z_nr_nouns'] = train['total_nouns'].map(lambda x: len(x))
    train['z_nr_verbs'] = train['total_verbs'].map(lambda x: len(x))
    train['z_nr_proper_nouns'] = train['total_proper_nouns'].map(lambda x: len(x))
    train['z_nr_cc'] = train['total_cc'].map(lambda x: len(x))
    train['z_nr_cd'] = train['total_cd'].map(lambda x: len(x))
    train['z_nr_dt'] = train['total_dt'].map(lambda x: len(x))
    train['z_nr_jj'] = train['total_jj'].map(lambda x: len(x))
    train['z_nr_rb'] = train['total_rb'].map(lambda x: len(x))
    train['z_nr_ex'] = train['total_ex'].map(lambda x: len(x))
    train['z_nr_ls'] = train['total_ls'].map(lambda x: len(x))
    train['z_nr_wdt'] = train['total_wdt'].map(lambda x: len(x))


    train['z_word_match'] = train.apply(word_match_share, axis=1, raw=True)
    train['z_tfidf_word_match'] = train.apply(tfidf_word_match_share, axis=1, raw=True)

    train["unigrams_ques1"] = train['question1'].apply(lambda x: get_unigrams(str(x)))
    train["unigrams_ques2"] = train['question2'].apply(lambda x: get_unigrams(str(x)))
    train["zunigrams_common_count"] = train.apply(lambda r: get_common_unigrams(r),axis=1)
    train["zunigrams_common_ratio"] = train.apply(lambda r: get_common_unigram_ratio(r), axis=1)
    train["bigrams_ques1"] = train["unigrams_ques1"].apply(lambda x: get_bigrams(x))
    train["bigrams_ques2"] = train["unigrams_ques2"].apply(lambda x: get_bigrams(x)) 
    train["zbigrams_common_count"] = train.apply(lambda r: get_common_bigrams(r),axis=1)


In [4]:
def feats_test():
    test['test'] = test.question1.map(lambda x: [t for  t in nltk.pos_tag(nltk.word_tokenize(str(x).lower()))])
    test['test1'] = test.question2.map(lambda x: [t for  t in nltk.pos_tag(nltk.word_tokenize(str(x).lower()))])

    test['question1_nouns'] = test.test.map(lambda x: [w for w, t in x if t == 'NN'])
    test['question2_nouns'] = test.test1.map(lambda x: [w for w, t in x if t == 'NN'])

    test['question1_proper_nouns'] = test.test.map(lambda x: [w for w, t in x if t[:3] in ['NNP']])
    test['question2_proper_nouns'] = test.test1.map(lambda x: [w for w, t in x if t[:3] in ['NNP']])

    test['question1_verbs'] = test.test.map(lambda x: [w for w, t in x if t[:2] in ['VB']])
    test['question2_verbs'] = test.test1.map(lambda x: [w for w, t in x if t[:2] in ['VB']])

    test['question1_jj'] = test.test.map(lambda x: [w for w, t in x if t[:2] in ['JJ']])
    test['question2_jj'] = test.test1.map(lambda x: [w for w, t in x if t[:2] in ['JJ']])

    test['question1_rb'] = test.test.map(lambda x: [w for w, t in x if t[:2] in ['RB']])
    test['question2_rb'] = test.test1.map(lambda x: [w for w, t in x if t[:2] in ['RB']])

    test['question1_cc'] = test.test.map(lambda x: [w for w, t in x if t == 'CC'])
    test['question2_cc'] = test.test1.map(lambda x: [w for w, t in x if t == 'CC'])

    test['question1_cd'] = test.test.map(lambda x: [w for w, t in x if t == 'CD'])
    test['question2_cd'] = test.test1.map(lambda x: [w for w, t in x if t == 'CD'])

    test['question1_dt'] = test.test.map(lambda x: [w for w, t in x if t == 'DT'])
    test['question2_dt'] = test.test1.map(lambda x: [w for w, t in x if t == 'DT'])

    test['question1_ex'] = test.test.map(lambda x: [w for w, t in x if t == 'EX'])
    test['question2_ex'] = test.test1.map(lambda x: [w for w, t in x if t == 'EX'])

    test['question1_ls'] = test.test.map(lambda x: [w for w, t in x if t == 'LS'])
    test['question2_ls'] = test.test1.map(lambda x: [w for w, t in x if t == 'LS'])

    test['question1_wdt'] = test.test.map(lambda x: [w for w, t in x if t == 'WDT'])
    test['question2_wdt'] = test.test1.map(lambda x: [w for w, t in x if t == 'WDT'])

    test['z_word_len1'] = test.question1.map(lambda x: len(str(x).split()))
    test['z_word_len2'] = test.question2.map(lambda x: len(str(x).split()))
    test['z_noun_match'] = test.apply(lambda r: sum([1 for w in r.question1_nouns if w in r.question2_nouns]), axis=1)
    test['z_verb_match'] = test.apply(lambda r: sum([1 for w in r.question1_verbs if w in r.question2_verbs]), axis=1)
    test['z_prop_noun_match'] = test.apply(lambda r: sum([1 for w in r.question1_proper_nouns if w in r.question2_proper_nouns]), axis=1)
    test['z_cc_match'] = test.apply(lambda r: sum([1 for w in r.question1_cc if w in r.question2_cc]), axis=1)
    test['z_cd_match'] = test.apply(lambda r: sum([1 for w in r.question1_cd if w in r.question2_cd]), axis=1)
    test['z_dt_match'] = test.apply(lambda r: sum([1 for w in r.question1_dt if w in r.question2_dt]), axis=1)
    test['z_jj_match'] = test.apply(lambda r: sum([1 for w in r.question1_jj if w in r.question2_jj]), axis=1)
    test['z_rb_match'] = test.apply(lambda r: sum([1 for w in r.question1_rb if w in r.question2_rb]), axis=1)
    test['z_ex_match'] = test.apply(lambda r: sum([1 for w in r.question1_ex if w in r.question2_ex]), axis=1)
    test['z_ls_match'] = test.apply(lambda r: sum([1 for w in r.question1_ls if w in r.question2_ls]), axis=1)
    test['z_wdt_match'] = test.apply(lambda r: sum([1 for w in r.question1_wdt if w in r.question2_wdt]), axis=1)



    #train['nr_nouns'] = train.question1.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    test['total_nouns'] = test['question1_nouns'] + test['question2_nouns']
    test['total_verbs'] = test['question1_verbs'] + test['question2_verbs']
    test['total_proper_nouns'] = test['question1_proper_nouns'] + test['question2_proper_nouns']
    test['total_cc'] = test['question1_cc'] + test['question2_cc']
    test['total_cd'] = test['question1_cd'] + test['question2_cd']
    test['total_dt'] = test['question1_dt'] + test['question2_dt']
    test['total_jj'] = test['question1_jj'] + test['question2_jj']
    test['total_rb'] = test['question1_rb'] + test['question2_rb']
    test['total_ex'] = test['question1_ex'] + test['question2_ex']
    test['total_ls'] = test['question1_ls'] + test['question2_ls']
    test['total_wdt'] = test['question1_wdt'] + test['question2_wdt']

    test['z_nr_nouns'] = test['total_nouns'].map(lambda x: len(x))
    test['z_nr_verbs'] = test['total_verbs'].map(lambda x: len(x))
    test['z_nr_proper_nouns'] = test['total_proper_nouns'].map(lambda x: len(x))
    test['z_nr_cc'] = test['total_cc'].map(lambda x: len(x))
    test['z_nr_cd'] = test['total_cd'].map(lambda x: len(x))
    test['z_nr_dt'] = test['total_dt'].map(lambda x: len(x))
    test['z_nr_jj'] = test['total_jj'].map(lambda x: len(x))
    test['z_nr_rb'] = test['total_rb'].map(lambda x: len(x))
    test['z_nr_ex'] = test['total_ex'].map(lambda x: len(x))
    test['z_nr_ls'] = test['total_ls'].map(lambda x: len(x))
    test['z_nr_wdt'] = test['total_wdt'].map(lambda x: len(x))


    test['z_word_match'] = test.apply(word_match_share, axis=1, raw=True)
    test['z_tfidf_word_match'] = test.apply(tfidf_word_match_share, axis=1, raw=True)

    test["unigrams_ques1"] = test['question1'].apply(lambda x: get_unigrams(str(x)))
    test["unigrams_ques2"] = test['question2'].apply(lambda x: get_unigrams(str(x)))
    test["zunigrams_common_count"] = test.apply(lambda r: get_common_unigrams(r),axis=1)
    test["zunigrams_common_ratio"] = test.apply(lambda r: get_common_unigram_ratio(r), axis=1)
    test["bigrams_ques1"] = test["unigrams_ques1"].apply(lambda x: get_bigrams(x))
    test["bigrams_ques2"] = test["unigrams_ques2"].apply(lambda x: get_bigrams(x)) 
    test["zbigrams_common_count"] = test.apply(lambda r: get_common_bigrams(r),axis=1)


In [5]:
def scale_data():
    global train, test, col
    train = train.fillna(-1)
    test = test.fillna(-1)

    col = [c for c in train.columns if c[:1]=='z']

    pos_train = train[train['is_duplicate'] == 1]
    neg_train = train[train['is_duplicate'] == 0]
    p = 0.165
    scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
    while scale > 1:
        neg_train = pd.concat([neg_train, neg_train])
        scale -=1
    neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
    train = pd.concat([pos_train, neg_train])
    
def save_processed_pickle():
    pd.to_pickle(train, "./data/train.pkl")
    pd.to_pickle(test, "./data/train.pkl")
    
def load_processed_pickle():
    global train, test
    print("Loading train and test pickles. Preprocessed versions.")
    train = pd.read_pickle("./data/train.pkl")
    test = pd.read_pickle("./data/test.pkl")


In [ ]:
try:
    load_processed_pickle()
except:
    print("Loading train and test pickles failed. Preprocessing from scratch :(")
    load_csv()
    common_words()
    feats_train()
    feats_test()
    scale_data

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:50: RuntimeWarning: invalid value encountered in double_scalars


modelling

In [ ]:
print()

In [69]:
x_train, x_valid, y_train, y_valid = train_test_split(train[col], train['is_duplicate'], test_size=0.2, random_state=0)


def score(params):
    global model, dvalid, dtrain, predictions
    
    print ("Training with params: ")
    print (params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_valid, label=y_valid)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid).reshape((dvalid.num_row(), 2))
    score = log_loss(y_valid, predictions)
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 2000, 1),
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.choice('max_depth', np.arange(1, 14, dtype=int)),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'num_class' : 2,
             'eval_metric': 'mlogloss',
             'objective': "binary:logistic", #"binary:logistic"
             'nthread' : 30,
             'silent' : 0
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=50)

    print (best)

#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

Training with params: 
{'silent': 0, 'n_estimators': 999.0, 'nthread': 30, 'eta': 0.25, 'eval_metric': 'mlogloss', 'gamma': 0.75, 'objective': 'multi:softprob', 'subsample': 0.7000000000000001, 'max_depth': 4, 'num_class': 2, 'colsample_bytree': 0.9500000000000001, 'min_child_weight': 6.0}
	Score 0.3170245902848997


Training with params: 
{'silent': 0, 'n_estimators': 199.0, 'nthread': 30, 'eta': 0.45, 'eval_metric': 'mlogloss', 'gamma': 0.6000000000000001, 'objective': 'multi:softprob', 'subsample': 0.65, 'max_depth': 6, 'num_class': 2, 'colsample_bytree': 0.9500000000000001, 'min_child_weight': 6.0}
	Score 0.31412337725477474


Training with params: 
{'silent': 0, 'n_estimators': 747.0, 'nthread': 30, 'eta': 0.07500000000000001, 'eval_metric': 'mlogloss', 'gamma': 0.6000000000000001, 'objective': 'multi:softprob', 'subsample': 0.8, 'max_depth': 8, 'num_class': 2, 'colsample_bytree': 0.7000000000000001, 'min_child_weight': 4.0}
	Score 0.30365744563114555


Training with params: 
{'si

KeyboardInterrupt: 

In [68]:
model.predict(dvalid)

array([[  4.88191992e-01,   5.11772692e-01,   5.04951186e-06, ...,
          5.04974787e-06,   5.04942500e-06,   5.05005573e-06],
       [  7.19469488e-01,   2.80500770e-01,   4.24165455e-06, ...,
          4.24185282e-06,   4.24158179e-06,   4.24211157e-06],
       [  9.99999762e-01,   2.12397268e-07,   8.22790547e-09, ...,
          8.22828206e-09,   8.22774915e-09,   8.22878476e-09],
       ..., 
       [  8.23094010e-01,   1.76879272e-01,   3.82033704e-06, ...,
          3.82051212e-06,   3.82027156e-06,   3.82074541e-06],
       [  6.76247358e-01,   3.23719233e-01,   4.77376761e-06, ...,
          4.77399044e-06,   4.77368576e-06,   4.77428193e-06],
       [  9.99890089e-01,   1.08783453e-04,   1.56321406e-07, ...,
          1.56328866e-07,   1.56318720e-07,   1.56338402e-07]], dtype=float32)

In [57]:
model.predict(dvalid).reshape((x_valid.shape[0], 2))

NameError: name 'dvalid' is not defined

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

rfr = ExtraTreesClassifier()

model1 = rfr.fit(train[col], train['is_duplicate'])


In [23]:
import time
t0 = time.time()
print("Parameter optimization")
xgb_model = xgb.XGBClassifier(seed=2017, objective="binary:logistic")
clf = GridSearchCV(xgb_model,
                   {'max_depth': [4,7], # 3, 5, 8
                    'n_estimators': [100,400], # 50,100,200
                    'learning_rate': [0.01, 0.3],
                    'min_child_weight': [0, 1, 1.5], # 1, 1.5, 5
                    'gamma': [0, 0.1], # 0, 0.1
                    'subsample': [0.5, 0.75, 1], # 0.5, 0.75, 1
                    'colsample_bytree': [0.5, 1], # 0.5, 0.75, 1,
                    #'eta': [0.05]
                   }, verbose=3, cv=3,scoring='neg_log_loss', n_jobs=-1)
clf.fit(train[col], train['is_duplicate'])
print(str(int(time.time() - t0)))
print("Best score:")
print(clf.best_score_)
print("Best parameters:")
print(clf.best_params_)

Parameter optimization
Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] gamma=0, learning_rate=0.01, subsample=0.5, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=0.5, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=0.5, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=0.75, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=0.75, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=0.75, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=1, max_depth=4, colsample_bytree=0.5, n_estimators=100, min_child_weight=0 
[CV] gamma=0, learning_rate=0.01, subsample=1, max_depth=4,

KeyboardInterrupt: 

In [72]:
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = 'logloss'
params["eta"] = 0.375 # 0.05
params["subsample"] = 0.75 # 1.0
params["min_child_weight"] = 3
params["colsample_bytree"] = 0.65 # 1.0
params["max_depth"] = 5 # 5
params["silent"] = 1
params["seed"] = 2017



d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 5000, watchlist, early_stopping_rounds=50, verbose_eval=100) 

[0]	train-logloss:0.536157	valid-logloss:0.5367
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[100]	train-logloss:0.321855	valid-logloss:0.3295
[200]	train-logloss:0.311755	valid-logloss:0.323944
[300]	train-logloss:0.304759	valid-logloss:0.321192
[400]	train-logloss:0.298762	valid-logloss:0.318791
[500]	train-logloss:0.293159	valid-logloss:0.316835
[600]	train-logloss:0.288136	valid-logloss:0.315121
[700]	train-logloss:0.283651	valid-logloss:0.313787
[800]	train-logloss:0.279235	valid-logloss:0.312433
[900]	train-logloss:0.275045	valid-logloss:0.311331
[1000]	train-logloss:0.270962	valid-logloss:0.309929
[1100]	train-logloss:0.267324	valid-logloss:0.30887
[1200]	train-logloss:0.263649	valid-logloss:0.307913
[1300]	train-logloss:0.260274	valid-logloss:0.306956
[1400]	train-logloss:0.256753	valid-logloss:0.305865
[1500]	train-logloss:0.253261	valid-logloss:0.304648
[1600]	train-loglo

In [73]:
d_test = xgb.DMatrix(test[col])
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = test['test_id']
sub['is_duplicate'] = p_test

#df['is_duplicate'] = df['is_duplicate'].map(lambda x: 0.000000000001 if x < 0.0001 else x)
#df['is_duplicate'] = df['is_duplicate'].map(lambda x: 0.999999999999 if x > 0.98 else x)

sub.to_csv('./submissions/submission_xgb_0424_aeron2.csv', index=False)
print(log_loss(train.is_duplicate, bst.predict(xgb.DMatrix(train[col]))))


0.196895026841


In [10]:
test.head()

,test_id,question1,question2,test,test1,question1_nouns,question2_nouns,question1_proper_nouns,question2_proper_nouns,question1_verbs,...,question1_cd,question2_cd,question1_dt,question2_dt,question1_ex,question2_ex,question1_ls,question2_ls,question1_wdt,question2_wdt
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[(how, WRB), (does, VBZ), (the, DT), (surface,...","[(why, WRB), (did, VBD), (microsoft, VB), (cho...","[surface, compare]","[core, m3, i3, home, surface]",[],[],"[does, pro]",...,[4],[4],[the],[],[],[],[],[],[],[]
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[(should, MD), (i, VB), (have, VBP), (a, DT), ...","[(how, WRB), (much, JJ), (cost, NN), (does, VB...","[hair, transplant, age]","[cost, require]",[],[],"[i, have, cost]",...,[24],[],[a],[],[],[],[],[],[],[]
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[(what, WP), (but, CC), (is, VBZ), (the, DT), ...","[(what, WP), (you, PRP), (send, VBP), (money, ...","[way, money, china]",[money],[],[],"[is, send]",...,[],[],"[the, the]",[],[],[],[],[],[],[]
3,3,Which food not emulsifiers?,What foods fibre?,"[(which, WDT), (food, NN), (not, RB), (emulsif...","[(what, WP), (foods, VBZ), (fibre, NN), (?, .)]",[food],[fibre],[],[],[],...,[],[],[],[],[],[],[],[],[which],[]
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[(how, WRB), (``, ``), (aberystwyth, NN), ('',...","[(how, WRB), (their, PRP$), (can, MD), (i, VB)...","[aberystwyth, start, reading]",[reading],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
